In [8]:
import dicom
from matplotlib import pyplot as plt, cm
import numpy as np
import cv2 
import os

%matplotlib inline

In [9]:
DATA_DIR = "/Volumes/homes/users/anthony.reina/dicom/Lung CT/stage1/"
    
subjects = os.listdir(DATA_DIR)

In [10]:
subjects

['0030a160d58723ff36d73f41b170ec21',
 '0092c13f9e00a3717fdc940641f00015',
 '00cba091fa4ad62cc3200a657aeb957e',
 '00edff4f51a893d80dae2d42a7f45ad1',
 '013395589c01aa01f8df81d80fb0e2b8',
 '.DS_Store']